In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
old_train_df = pd.read_csv('/root/tempfile/train_final122002.csv')
test_df = pd.read_csv('/root/tempfile/test_final122002.csv')

In [3]:
train_df = old_train_df[old_train_df.target>-30]

In [4]:
print(old_train_df.shape, test_df.shape)

(201917, 265) (123623, 264)


In [5]:
feature_col = [x for x in train_df.columns if x not in ['card_id','target']]
y_train = train_df.target.values.astype('float')
x_train = train_df[feature_col].values.astype('float')
x_test = test_df[feature_col].values.astype('float')

In [6]:
testindex = test_df.card_id

In [7]:
del train_df, test_df, old_train_df

In [8]:
lgb_model = lgb.LGBMRegressor(objective='regression',
                              min_data_in_leaf=20,
                              num_leaves=50,
                              learning_rate=0.1, 
                              n_estimators=43, 
                              max_depth=8,
                              metric='mse',
                              bagging_fraction = 0.8,
                              feature_fraction = 0.8,
                              num_threads=2)

In [9]:
params_test1={
    'max_depth': range(8,11,2),
    'num_leaves':range(100, 300, 20),
    'min_data_in_leaf':range(100,300,20)
}
gsearch1 = GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test1,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=4)

gs1 = gsearch1.fit(x_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  4.4min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 19.0min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 42.8min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 83.4min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed: 107.7min finished


In [10]:
print(gs1.best_params_, gs1.best_score_)

{'max_depth': 10, 'min_data_in_leaf': 240, 'num_leaves': 160} -2.4339175243361613
